def thesis_expt_function(feat_sel_technique, model, dataset):
    '''
    takes in two function handles and returns a dataframe or series with one column per metric 

    Parameters: 
   ---------------
    feat_sel_technique: xy_greater_xa, maximally_demographic_informative_subset, or MRMR
    model : estimator object 
    dataset: csv file 

    Return 
-------- 
    resutl_df : one column per metric
    '''

In [1]:
# Requirements
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.metrics import mutual_info_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn import metrics
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
from expttools import Experiment

# Base Model

In [3]:

def categorical_feature_encoder(data,features):
    '''
    takes a data frame and returns numerical encoding for categorical features
    
    Parameters: 
    ----------- 
    dataset: csv file 
    features : list of categorical features

    Return 
    ------
    returns : dataframe with encoded features, encoding for categorical features  
    '''
    enc = {}
    
    for f in features:
        encoder = OrdinalEncoder()
        data[f] = encoder.fit_transform(data[[f]]).astype(int)
        enc[f] = encoder
    return data, enc



In [7]:

def df_manipulation(features_encoded_data,data,col_name,priviliged_vals):
    '''
    takes encoded data frame and original data to substitute an original column from original df to 
    encoded df for further analysis
    
    Parameters: 
    ----------- 
    features_encoded_data: encoded dataset 
    data : original data
    col_name : column for manipulation. eg: if comparison is done between black and white, this piece 
    of code will remove all other 
    races and return black as 0 white as 1 and 

    Return 
    ------
    returns : dataframe with encoded features containing manipulated column 
    '''
    
    decoded_col_name = f'{col_name}_decoded'
    filtered_col_name = f'filtered_{col_name}'
    
    #priviliged_vals = ['Married-civ-spouse', 'Married-spouse-absent']
    features_encoded_data[decoded_col_name] = data[col_name]
    features_encoded_data[filtered_col_name] = features_encoded_data[decoded_col_name]\
    .isin(priviliged_vals).astype(int)

    features_encoded_data = features_encoded_data.drop([decoded_col_name], axis = 1)

    return features_encoded_data


In [8]:

def base_model(data,target,col_name):
    '''
    takes feature encoded data, splits for training and test set and returns the data frame with predictions.
    
    Parameters: 
    ----------- 
    data : encoded data
    target : proxy target
    col_name : column for manipulation. 

    Return 
    ------
    returns : original_output -- predicted set
    '''
    encoded_df = data.copy()
    x = encoded_df.drop([target], axis = 1)
    y = encoded_df[target]
    filtered_col_name = f'filtered_{col_name}'
    #print(filtered_col_name)
    
    x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2, random_state = 0)
    x_train[filtered_col_name] = x_train[filtered_col_name].apply(lambda x:1 if x>0 else 0)
    x_test[filtered_col_name] = x_test[filtered_col_name].apply(lambda x:1 if x>0 else 0)
    sc = StandardScaler()
    x_train = pd.DataFrame(sc.fit_transform(x_train),columns = x_train.columns)
    x_test = pd.DataFrame(sc.transform(x_test),columns = x_test.columns)
    x_train[filtered_col_name] = x_train[filtered_col_name].apply(lambda x:1 if x>0 else 0)
    x_test[filtered_col_name] = x_test[filtered_col_name].apply(lambda x:1 if x>0 else 0)
    classifier = LogisticRegression()
    classifier.fit(x_train, y_train)
    # We now need to add this array into x_test as a column for when we calculate the fairness metrics.
    y_pred = classifier.predict(x_test)
    x_test['target_predicted'] = y_pred
    original_output = x_test
    original_output['actual'] = y_test.values
    return original_output
    

In [9]:
def get_fairness_metrics_bc(original_output,col_name):
    '''
    takes prediction df and returns calculated fairness metrics 
    
    Parameters: 
    ----------- 
    data : original_output with prediction column
    col_name : column for manipulation. 

    Return 
    ------
    returns : fairness metrics
    '''
    filtered_col_name = f'filtered_{col_name}'

    male_df = original_output[original_output[filtered_col_name] == 1]
    num_of_priviliged = male_df.shape[0]
    female_df = original_output[original_output[filtered_col_name] == 0]
    num_of_unpriviliged = female_df.shape[0]

    unpriviliged_outcomes = female_df[female_df['target_predicted'] == 1].shape[0]
    unpriviliged = unpriviliged_outcomes/num_of_unpriviliged
    unpriviliged

    priviliged_outcomes = male_df[male_df['target_predicted'] == 1].shape[0]
    priviliged = priviliged_outcomes/num_of_priviliged
    priviliged

    #Disparate impact
    disparate_impact = unpriviliged / priviliged

    #Statistical parity difference 
    statistical_parity_difference  = unpriviliged - priviliged
    
    #Equal opportunity difference
    eod = original_output.copy()
    eod = eod[eod['actual'] == 1] 
    eod ['true_positives'] = eod ['target_predicted'] == eod['actual']

    eod_other = eod[eod[filtered_col_name]== 0]['true_positives'].mean()

    eod_married_civ_absent = eod[eod[filtered_col_name] == 1]['true_positives'].mean()
    equal_opportunity_difference  = eod_other - eod_married_civ_absent
    
    #accuracy
    accuracy = (original_output['target_predicted']== original_output['actual']).mean()
    
    return ([disparate_impact,statistical_parity_difference,equal_opportunity_difference,accuracy])

# Fair Model

In [10]:
# Requirements
from aif360.algorithms.inprocessing import PrejudiceRemover
from aif360.metrics import ClassificationMetric
from sklearn.metrics import accuracy_score
from aif360.datasets import AdultDataset
from sklearn.preprocessing import StandardScaler
from aif360.datasets import StructuredDataset, BinaryLabelDataset
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

2022-09-13 13:59:24.022277: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-13 13:59:24.402719: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /modules/apps/julia/1.7.2/lib
2022-09-13 13:59:24.402742: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
def categorical_feature_encoder(data,features):
    '''
    takes a data frame and returns numerical encoding for categorical features
    
    Parameters: 
    ----------- 
    dataset: csv file 
    features : list of categorical features

    Return 
    ------
    returns : dataframe with encoded features, encoding for categorical features  
    '''
    enc = {}
    
    for f in features:
        encoder = OrdinalEncoder()
        data[f] = encoder.fit_transform(data[[f]]).astype(int)
        enc[f] = encoder
    return data, enc

#feature_encoded_data = categorical_feature_encoder(data, features)

In [12]:
def fair_model(data, subset_cols, target, p_att):
    
    '''
    takes encoded df and formats into a binary label df format to split and return test and train df.
    
    Parameters: 
    ----------- 
    dataset: original data 
    subset_cols : columns from dictionary generated from feature selection technique
    target : proxy target
    p_att: single protected attribute, key from dictionary generated from feature selection technique
    
    Return 
    ------
    returns : dataset_orig_train, dataset_orig_test -- test and trained datasets 
    '''
    
    #subset_sex_cols = ['education-num', 'capital-gain', 'capital-loss', 'hours-per-week','education', 'native-country', 'workclass','sex','income-per-year']
    encoded_df = data.copy()
    structured_data = BinaryLabelDataset(favorable_label=1.0, unfavorable_label=0.0, df = encoded_df[subset_cols]\
                                         .dropna(), label_names = [target], protected_attribute_names = [p_att], \
                                         instance_weights_name=None, scores_names=[], unprivileged_protected_attributes\
                                         =[[0]], privileged_protected_attributes=[[1]], metadata=None)
    dataset_orig = structured_data
    privileged_groups = [{p_att: 1}] #male 
    unprivileged_groups = [{p_att: 0}] #female

    dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)
    scaler = StandardScaler()
    dataset_orig_train.features = scaler.fit_transform(dataset_orig_train.features)
    dataset_orig_test.features = scaler.transform(dataset_orig_test.features) 
    return dataset_orig_train,dataset_orig_test

#dataset_orig_train,dataset_orig_test = fair_model(data, subset_cols, target, p_att)


In [2]:
def get_fairness_metrics_fc(dataset_orig_train,dataset_orig_test,p_att,privileged_groups,unprivileged_groups,etas):
    
    '''
    takes train and test dataset runs using PrejudiceRemover classifier and returns fairness metrics
    
    Parameters: 
    ----------- 
    dataset_orig_train: training data set 
    dataset_orig_test: test dataset
    p_att: single protected attribute, chooses particular key from dictionary generated from feature selection technique
    privileged_groups : [1]
    unprivileged_groups : [0]
    etas : fairness penalty parameter
    
    Return 
    ------
    returns : fairness metrics 
    '''
    
    #etas = [1]
    outputs = []
    #print(dataset_orig_train,dataset_orig_test,p_att,privileged_groups,unprivileged_groups,etas)
    
    for eta in etas:
        debiased_model = PrejudiceRemover(eta=eta, sensitive_attr = p_att, class_attr=dataset_orig_train.label_names[0])        
        model = debiased_model.fit(dataset_orig_train)
        pred = model.predict(dataset_orig_test)

        metric = ClassificationMetric(dataset_orig_test, pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
        
        outputs.append([eta,metric.disparate_impact(),metric.statistical_parity_difference(),metric.equal_opportunity_difference(),\
                        accuracy_score(pred.labels, dataset_orig_test.labels)])
    return outputs

#dataset_orig_train,dataset_orig_test, filtered_col,privileged_groups = [{filtered_col:1}] ,unprivileged_groups = [{filtered_col:0}],etas=etas


### Feature selection technique - XY > XA : this technique generates dictionary with keys as protected att and values as features (all features where XY>XA for the particular protected att). We use the values (features) as subsets of features

In [3]:
#requirements
import pandas as pd
import numpy as np
from sklearn.metrics import mutual_info_score
from sklearn.preprocessing import OrdinalEncoder

In [21]:
# features segregation 
features_num = ['hours-per-week', 'capital-gain', 'capital-loss','education-num' ] 
features_cat =  ['workclass','education', 'relationship', 'occupation', 'native-country'] 
protected_attributes = ['sex','race','age', 'marital-status']
target = 'income-per-year'

In [5]:
#encoding the categorical columns, OrdinalEncoder - each unique category value is assigned an integer value
def categorical_feature_encoder(data,features):
    '''
    takes a data frame and returns numerical encoding for categorical features
    
    Parameters: 
    ----------- 
    dataset: csv file 
    features : list of categorical features

    Return 
    ------
    returns : dataframe with encoded features, encoding for categorical features  
    '''
    enc = {}
    
    for f in features:
        encoder = OrdinalEncoder()
        data[f] = encoder.fit_transform(data[[f]]).astype(int)
        enc[f] = encoder
    return data, enc

In [7]:
#function for calculating mutual information score for each X (features) and Y (target)
def calculate_miscore_xy(data, y_col,a_col):
    '''
    takes a data frame and returns a data frame with mutual information score between X(features) and Y(target)
    
    Parameters: 
    ----------- 
    dataset: csv file 
    y_col : target column
    a_col : protected attributes 
    
    Return 
    ------
    returns : returns mutual information score between X(features) and Y(target) in a dataframe 
    '''
    mis_xy = []
    x_cols = []
    for x in data.columns:
        if not (x in a_col): # skipping the demographic features 
            mis = mutual_info_score(data[x], data[y_col], contingency=None) # mis calculation
            mis_xy.append(mis) 
            x_cols.append(x)


    adult_dataFrame_feature_target = pd.DataFrame({'I(Xi,Y)': mis_xy, 'X': x_cols}) # creating pandas dataframe
    adult_dataFrame_feature_target = adult_dataFrame_feature_target.loc[adult_dataFrame_feature_target['X'] != y_col]#loc : filtering dataframe based on index
    adult_dataFrame_feature_target['Y'] = y_col # adding y column
    return adult_dataFrame_feature_target


In [8]:
#function for calculating mutual information score for each X (features) and A (demographic variables)

def calculate_miscore_xa(data,protected_attributes):
    '''
    takes a data frame and returns a data frame with mutual information score between X(features) and A(protected attributes)
    
    Parameters: 
    ----------- 
    dataset: csv file 
    protected_attributes : protected attributes 
    
    Return 
    ------
    returns : returns mutual information score between X(features) and A(protected attributes) in a dataframe 
    '''
    
    mis_xa = []
    attribute_unfiltered = []
    feature_unfiltered = []
    for x in data.columns:
        for a in protected_attributes:
            if not (x in protected_attributes):
                mis = mutual_info_score(data[a], data[x], contingency=None)
                mis_xa.append(mis)
                attribute_unfiltered.append(a)  
                feature_unfiltered.append(x)

    unfiltered_mis_adult_dataFrame = pd.DataFrame({'X':feature_unfiltered, 'A':attribute_unfiltered, 'I(Xi,A)': mis_xa})
    return unfiltered_mis_adult_dataFrame


In [10]:
#merging dataframes
def generate_xy_greater_xa(data1,data2):
    '''
    takes two data frames (in our case two df generated for I(X,Y) and I(X,A) and returns a dictionary with keys as protected att and values as features 
    (features = all features where XY>XA for a particular protected att)
    
    Parameters: 
    ----------- 
    data1: dataframe 
    data2: dataframe
    
    Return 
    ------
    returns : dictionary with key as p attributes and values as features
    '''
    merged_xiY_xiA = pd.merge(data1,data2, on=['X'], how = 'inner') 
    merged_xiY_xiA
    merged_xiY_xiA.to_csv('merged_xiY_xiA.csv')
    #adding a new bool column, True if xi_A > xi_Y
    merged_xiY_xiA['X_sub_A = True'] = merged_xiY_xiA['I(Xi,A)'] > merged_xiY_xiA['I(Xi,Y)']

    # for each val of A pick list of features where XY>XA - X_sub_A¯
    dictonary_xy_greaterthan_xa = {} 
    for a_v, df_a in merged_xiY_xiA.groupby('A'):
        dictonary_xy_greaterthan_xa[a_v] = df_a[df_a['X_sub_A = True']== False]['X'].values

    return dictonary_xy_greaterthan_xa


# Building main driver function for base and fair classifier

In [36]:
#reading dataset
adult_dataset = pd.read_csv('adult_dataset.csv')
adult_dataset.drop('Unnamed: 0', axis = 1, inplace = True)
adult_dataset.head()

,age,education-num,sex,capital-gain,capital-loss,hours-per-week,income-per-year,education,marital-status,native-country,occupation,race,relationship,workclass
0,25.0,7.0,1.0,0.0,0.0,40.0,0.0,11th,Never-married,United-States,Machine-op-inspct,Black,Own-child,Private
1,38.0,9.0,1.0,0.0,0.0,50.0,0.0,HS-grad,Married-civ-spouse,United-States,Farming-fishing,White,Husband,Private
2,28.0,12.0,1.0,0.0,0.0,40.0,1.0,Assoc-acdm,Married-civ-spouse,United-States,Protective-serv,White,Husband,Local-gov
3,44.0,10.0,1.0,7688.0,0.0,40.0,1.0,Some-college,Married-civ-spouse,United-States,Machine-op-inspct,Black,Husband,Private
4,34.0,6.0,1.0,0.0,0.0,30.0,0.0,10th,Never-married,United-States,Other-service,White,Not-in-family,Private


### Model functions for selecting a particular model -- both can be called in main function by specifying model type

In [12]:
def base_classifier(features_encoded_data,target,p_att_col,model_cols,filtered_col, etas): # etas not in use but passing for consistency of format
    '''
    takes encoded df, runs logistic regression on the data to return a df consiting of fairness metrics and model type column.
    
    Parameters: 
    ----------- 
    features_encoded_data: encoded dataframe from categorical_feature_encoder function 
    target: proxy target
    p_att_col : protected attributes in the data
    model_cols : specific subset used from the data for analysis
    filtered_col : used for filtering columns based on conditions eg. keeping black(0) and white(1) in race column and removing other races.
    etas : fairness penalty parameter, not used in base classifier
    
    Return 
    ------
    returns : fairness metric dataframe 
    '''
    original_output = base_model(features_encoded_data[model_cols].copy(),target,p_att_col)
    base_classifier_fairness_metrics = get_fairness_metrics_bc(original_output,p_att_col)
    return pd.DataFrame([['base_model']+base_classifier_fairness_metrics],columns = ['model_type','disparate_impact','statistical_parity_difference',\
                                                                                     'equal_opportunity_difference','accuracy'])


In [13]:
def fair_classifier(features_encoded_data,target,p_att_col,model_cols, filtered_col, etas):
    '''
    takes encoded df, runs prejudice remover on the data to return a df consiting of fairness metrics and model type column.
    
    Parameters: 
    ----------- 
    features_encoded_data: encoded dataframe from categorical_feature_encoder function 
    target: proxy target
    p_att_col : protected attributes in the data
    model_cols : specific subset used from the data for analysis
    filtered_col : used for filtering columns based on conditions eg. keeping black(0) and white(1) in race column and removing other races.
    etas : fairness penalty parameter
    
    Return 
    ------
    returns : fairness metric dataframe 
    '''
    dataset_orig_train,dataset_orig_test = fair_model(features_encoded_data, model_cols, target, filtered_col)
    fair_classifier_fairness_metrics = get_fairness_metrics_fc(dataset_orig_train,dataset_orig_test, filtered_col,privileged_groups = [{filtered_col:1}] ,\
                                                               unprivileged_groups = [{filtered_col:0}],etas=etas)
    result = pd.DataFrame(fair_classifier_fairness_metrics,columns = ['eta','disparate_impact','statistical_parity_difference','equal_opportunity_difference',\
                                                                      'accuracy'])
    result.insert(0,'model_type','fair_model')
    return result


In [14]:
def build_model(data,features_encoded_data, p_att_col, dict_subsets_xy_greaterthan_xa,target, priviliged_vals, model_type,etas):
    '''
    build model takes encoded data and handles the data manipulation part prior to modeling phase (used for both fair and base classifier) and 
    finally runs the model and returns output dataframe with fairness metrics
    
    Parameters: 
    ----------- 
    data : original dataframe
    features_encoded_data: encoded dataframe from categorical_feature_encoder function 
    target: proxy target
    p_att_col : protected attributes in the data
    dict_subsets_xy_greaterthan_xa : generated from generate_xy_greater_xa function
    target : proxy target
    priviliged_vals : priviliged value in a particular protected att, which should be considered as priviliged group. eg. Whites in races can be 
    considered as priviliged
    model_type : fair or base models 
    etas : fairness penalty parameter for fair model - prejudice remover
    
    Return 
    ------
    returns : dataframe 
    '''
    filtered_col = f'filtered_{p_att_col}'
    model_cols = [filtered_col] + list(dict_subsets_xy_greaterthan_xa[p_att_col]) + [target]
    features_encoded_data = df_manipulation(features_encoded_data,data,p_att_col,priviliged_vals)
    output = model_type(features_encoded_data,target,p_att_col,model_cols,filtered_col, etas)
    return output


### feature selection technique -- called in main func

In [15]:
def xy_greater_xa(data,features_cat,protected_attributes, target):
    '''
    takes data and returns encoded df and a dictionary with keys as protected att and values as features 
    (features = all features where XY>XA for a particular protected att) 
    
    Parameters: 
    ----------- 
    data : original dataframe
    features_cat: list of categorical features in data 
    protected_attributes : list of protected attributes in data
    target: proxy target
    
    Return 
    ------
    returns : encoded dataframe and a dictionary
    '''
    features_encoded_data,enc = categorical_feature_encoder(data.copy(),features_cat + protected_attributes)

    mi_Xi_Y = calculate_miscore_xy(features_encoded_data, target, protected_attributes)
    mi_Xi_A = calculate_miscore_xa(features_encoded_data,protected_attributes)
    dict_subsets_xy_greaterthan_xa = generate_xy_greater_xa(mi_Xi_Y,mi_Xi_A) 
    return features_encoded_data, dict_subsets_xy_greaterthan_xa


### main func

In [16]:
def main_exp_bf_func(data,features_cat,protected_attributes, target, p_att_col, priviliged_vals, etas, technique = xy_greater_xa, \
                     model = fair_classifier):
   
    '''
    takes data and other arguments to return fairness metrics for a particular model and feature selection technique 
    
    Parameters: 
    ----------- 
    data : original dataframe
    features_cat: list of categorical features in data 
    protected_attributes : list of protected attributes in data
    target: proxy target
    p_att_col : single protected attribute we want an analysis for
    priviliged_vals : privileged values for feature mentioned in p_att_col
    etas : fairness penalty parameter
    technique : feature selection technique used for subset generation
    model : estimator object, fair or base classifier
    
    Return 
    ------
    returns : dataframe with model type column and fairness metrics
    '''
    features_encoded_data, dict_subsets_xy_greaterthan_xa = technique(data,features_cat,protected_attributes, target)
    '''
    notes:
    - split the data here : couldn't do it here as both models have different ways for split (fair classifier is first conversted to binary label dataset and 
    then it's split into train and test df, it differes from the process of logistic regression)
    - instantiate the model object : 
    - fit : even fit differs so I did this part individually for both
    '''
    return build_model(data,features_encoded_data, p_att_col, dict_subsets_xy_greaterthan_xa,target, priviliged_vals, model, etas)
     


### Arguments to be passed in main function

In [38]:
features_num = ['hours-per-week', 'capital-gain', 'capital-loss','education-num' ] # all the numerical fearures
features_cat =  ['workclass','education', 'relationship', 'occupation', 'native-country'] # categorical features
protected_attributes = ['sex','race','age', 'marital-status'] # protected attributes
target = 'income-per-year' # proxy target
p_att_col = 'sex' # calculating on individual p_att 'sex'
priviliged_vals = [1] # [priviliged_vals = [white] for white vs all ; priviliged_vals = [white,asian] if both are considered privileged]
model = fair_classifier # it can be fair or base model
technique = xy_greater_xa # geature selection technique
etas = [1]
# import feature selection technique to main func - done
# calling main function with model type, technique and other args -  done

main_exp_bf_func(adult_dataset,features_cat,protected_attributes, target, p_att_col, priviliged_vals , etas, technique, model)
    

,model_type,eta,disparate_impact,statistical_parity_difference,equal_opportunity_difference,accuracy
0,fair_model,1,0.146126,-0.148876,-0.222169,0.809538


In [39]:
features_num = ['hours-per-week', 'capital-gain', 'capital-loss','education-num' ] 
features_cat =  ['workclass','education', 'relationship', 'occupation', 'native-country'] 
protected_attributes = ['sex','race','age', 'marital-status']
target = 'income-per-year'
p_att_col = 'sex'
priviliged_vals = [1] # [white vs all] [white,asian]
model = base_classifier
technique = xy_greater_xa
etas = [1]
# import feature selection technique to main func
# calling main function with model type, technique and other args

main_exp_bf_func(adult_dataset,features_cat,protected_attributes, target, p_att_col, priviliged_vals , etas, technique, model)

filtered_sex


,model_type,disparate_impact,statistical_parity_difference,equal_opportunity_difference,accuracy
0,base_model,0.179197,-0.153427,-0.223399,0.808845


## Exp tool

I tried doing this part but files being generated are marked as failed. Can you please check?

In [49]:
#debiased_model = PrejudiceRemover(eta=eta, sensitive_attr = p_att, class_attr=dataset_orig_train.label_names[0])
features_num = ['hours-per-week', 'capital-gain', 'capital-loss','education-num' ] 
features_cat =  ['workclass','education', 'relationship', 'occupation', 'native-country'] 
protected_attributes = ['sex','race','age', 'marital-status']
target = 'income-per-year'
p_att_col = 'sex'
priviliged_vals = [1] # [white vs all] [white,asian]
model = base_classifier
technique = xy_greater_xa
etas = [1]


thesis_param_grid = {'data':[adult_dataset],'features_cat': [features_cat], 'protected_attributes':[protected_attributes],\
                     'target': [target], 'p_att_col': [p_att_col],'priviliged_vals':[priviliged_vals] , 'etas': [etas], \
                     'technique': [technique], 'model': [model]}


In [50]:
my_expt = Experiment(main_exp_bf_func,thesis_param_grid)
batchname, successes ,fails = my_expt.run_batch()

In [44]:
import os
os.getcwd()

'/work/pi_brownsarahm_uri_edu/surbhi_uri/var_selection'

In [51]:
#os.mkdir('results')